# Format the resulting edges for import to neo4j and get files needed for learn pipeline

In [1]:
import json
import pandas as pd
import sys
sys.path.append('../py')
import processing as pt

In [2]:
edges = pd.read_hdf('data/all_edges_unfiltered.h5', 'edges')

## The Current Wikidata Metgraph

![Metagraph](wikidata-metagraph-0.2-cutoff.png)

After careful examination of this metagraph, it appears that there will be two major issues that contriube to an increased computational complexity:

1. The splitting of Genes into 4 different node types, all with the same paths, creating 8 edge types from 2 initally, adds produces a great number of combinations that will greatly increase the number of queries needed to be performed. 

2. The splitting of Proteins and Genes, that other knowledge graphs of this type, will require metapaths of length 5 to describe relationships that a metapath of length 4 would otherwise describe.

Because all proteins have a direct link to a Gene, this infomration doesn't needed to be captured by an edge.

We will do two things:

1. Collapse all genes into 1 type called 'Gene'

2. Replace all references to protiens with the id for their corresponding gene.  Will will, however, keep the label for the gene

### Collapse all genes into Gene

In [3]:
for kind in ['Gene', 'Protein-coding Gene', 'Pseudo Gene', 'Non-coding RNA']:
    edges.loc[edges['type']==kind, 'type'] = 'Gene'
edges['type'].value_counts()

Protein                            513942
Mature MicroRNA                    329845
Gene                               308917
Compound                           192147
GO Term                             60459
Protein Family                      36090
Disease                             28878
Protein Domain                      21819
Chemical Hazard                     17395
Supersecondary Structure             2690
Structural Motif                      588
Active Site                           539
Binding Site                          395
Symptom                               246
Post-translational Modification        72
Name: type, dtype: int64

### Create Gene to Protein Mapping

In [4]:
# Get all proteins
idxp = edges['type'] == 'Protein'
proteins = edges.loc[idxp, 's'].unique()
gpdf = edges.query('o in @proteins and pLabel == "encodes"')

In [5]:
print('Unique Proteins: ', gpdf['o'].nunique())
print('Unique Genes: ', gpdf['s'].nunique())

Unique Proteins:  25056
Unique Genes:  20099


Looks like we're going to lose about 5000 proteins, due to one-to-many gene-encodes-protein relationships

In [6]:
# Protein to Gene URI Mapping
p_to_g = gpdf.set_index('o')['s'].to_dict()

In [7]:
gpdf = edges.query('o in @proteins and pLabel == "encodes"')
idx_dup = gpdf.duplicated(subset='sLabel', keep=False)

# Begin with a mapping of gene to gene Label
g_to_pl = edges[edges['type'] == 'Gene'].drop_duplicates(subset='s').set_index('s')['sLabel'].to_dict()

# Update mappings with Proteins and Genes with 1-1 relationships that can be mapped the protein's name
g_to_pl.update(gpdf.loc[~idx_dup].set_index('s')['oLabel'].to_dict())

### Do the mapping

In [8]:
# Drop Gene-encodes-Protein Edges
gep_idx = edges.query('o in @proteins and pLabel == "encodes"').index
edges.drop(gep_idx, inplace=True)
print(len(edges))
# Drop Protein-encoded_by-Gene Edges
pebg_idx = edges.query('s in @proteins and pLabel == "encoded by"').index
edges.drop(pebg_idx, inplace=True)
print(len(edges))

# Reset the index and grab the remaining proteins that haven't been dropped
edges.reset_index(drop=True, inplace=True)
idxp = edges['type'] == 'Protein'

# Rename the types to gene, then apply the uri mappings
edges.loc[idxp, 'type'] = 'Gene'

edges.loc[idxp, 's'] = edges.loc[idxp, 's'].apply(lambda s: p_to_g.get(s, float('nan')))
edges['o'] = edges['o'].apply(lambda o: p_to_g.get(o, o))

# Remove any unmappable proteins
edges = edges.dropna()
print(len(edges))

# Apply Label Mappings
idxg = edges['type'] == 'Gene'
genes = edges.loc[idxg, 's'].unique()
idx_obj_g = edges.query('o in @genes').index

edges.loc[idxg, 'sLabel'] = edges.loc[idxg, 's'].apply(lambda s: g_to_pl[s])
edges.loc[idx_obj_g, 'oLabel'] = edges.loc[idx_obj_g, 'o'].apply(lambda o: g_to_pl[o])

print('Done')

1367881
1300192
1282791
Done


In [9]:
gndf = edges.query('type == "Gene"').drop_duplicates(subset = ['s', 'sLabel'])
gndf[gndf['s'].duplicated(keep=False)].sort_values('s')

,sLabel,pLabel,oLabel,s,p,o,type


In [10]:
edge_counts = edges['type'].value_counts()
node_counts = edges.groupby('type')['s'].nunique().sort_values(ascending=False)

avg_degree = edge_counts / node_counts

In [11]:
avg_degree.sort_values(ascending = False)

Mature MicroRNA                    111.919985
Chemical Hazard                     20.238026
Gene                                11.671057
Symptom                              5.514286
Binding Site                         4.243243
Active Site                          3.540984
Post-translational Modification      3.411765
Supersecondary Structure             3.293313
Disease                              2.766433
GO Term                              2.425202
Protein Domain                       2.188804
Structural Motif                     1.918699
Protein Family                       1.629478
Compound                             1.208826
dtype: float64

## Now that the edges have changed, re-examine the effect of different cutoff values for removing low count edge types

In [12]:
# Node types should return the same every time, so won't keep separate copies
p0, nt, r0 = pt.prep_for_export(edges, 0.5, 0)

In [13]:
p01, nt, r01 = pt.prep_for_export(edges, 0.5, 0.01)

In [14]:
p05, nt, r05 = pt.prep_for_export(edges, 0.5, 0.05)

In [15]:
p1, nt, r1 = pt.prep_for_export(edges, 0.5, 0.1)

In [16]:
p15, nt, r15 = pt.prep_for_export(edges, 0.5, 0.15)

In [12]:
p2, nt, r2 = pt.prep_for_export(edges, 0.5, 0.2)

In [18]:
p25, nt, r25 =  pt.prep_for_export(edges, 0.5, 0.25)

In [19]:
p3, nt, r3 = pt.prep_for_export(edges, 0.5, 0.3)

In [20]:
p5, nt, r5 = pt.prep_for_export(edges, 0.5, 0.5)

In [21]:
e0 = set(p0['e_type'])
e01 = set(p01['e_type'])
e05 = set(p05['e_type'])
e1 = set(p1['e_type'])
e15 = set(p15['e_type'])
e2 = set(p2['e_type'])
e25 = set(p25['e_type'])
e3 = set(p3['e_type'])
e5 = set(p5['e_type'])


for i, j in zip([e0, e01, e05, e1, e15, e2, e25, e3, e5], [0, .01, .05, .1, .15, .2, .25, .3, .5]):
    print('Cutoff: {}\tEdge Tyeps: {}'.format(j, len(i)))

Cutoff: 0	Edge Tyeps: 149
Cutoff: 0.01	Edge Tyeps: 59
Cutoff: 0.05	Edge Tyeps: 47
Cutoff: 0.1	Edge Tyeps: 44
Cutoff: 0.15	Edge Tyeps: 42
Cutoff: 0.2	Edge Tyeps: 37
Cutoff: 0.25	Edge Tyeps: 37
Cutoff: 0.3	Edge Tyeps: 35
Cutoff: 0.5	Edge Tyeps: 33


In [22]:
def print_cutoff_info(this_cut, name, prev_cut=None, prev_name=None):
    this_edge = set(this_cut['e_type'])
    if prev_name:
        prev_edge = set(prev_cut['e_type'])
    
    
    print('--{}--'.format(name))
    print('Total Number of Edge Types: {}'.format(len(set(this_cut['e_type']))))
    print('Edge Types with only 1 instance: {}'.format(sum(this_cut['e_type'].value_counts() <= 1)))
    print('Edge Types with 10 or fewer instances: {}'.format(sum(this_cut['e_type'].value_counts() <= 10)))

    if prev_name:
        print('\n')
        print('-'*50)
        print('This cutoff removes the following edges that were in {}:'.format(prev_name))
        print('-'*50)

        for edge in (prev_edge - this_edge):
            print(edge)
    

In [23]:
print_cutoff_info(p0, 'No Cuttoff')

--No Cuttoff--
Total Number of Edge Types: 149
Edge Types with only 1 instance: 47
Edge Types with 10 or fewer instances: 90


Without filtering, 149 edge types is A lot. 47 edge types with just a single edge is huge, and even more (about another 40) with fewer than 10

In [24]:
print_cutoff_info(p01, '0.01', p0, 'No Cutoff')

--0.01--
Total Number of Edge Types: 59
Edge Types with only 1 instance: 6
Edge Types with 10 or fewer instances: 10


--------------------------------------------------
This cutoff removes the following edges that were in No Cutoff:
--------------------------------------------------
facet-of_GTfoGT
physically-interacts-with_CpiwG
drug-used-for-treatment_CduftD
follows_GTfGT
different-from_CdfC
said-to-be-the-same-as_DstbtsaD
symptoms_DsGT
has-part_ChpSS
physically-interacts-with_GpiwG
medical-treatment_CHmtC
part-of_SSpoSS
has-part_PFhpG
part-of_CpoGT
facet-of_DfoD
has-contributing-factor_DhcfD
cause-of_DcoD
cell-component_CccGT
medical-treatment_DmtC
encoded-by_GebG
subclass-of_CsoC
gene-duplication-association-with_GgdawD
biological-process_PFbpGT
part-of_ASpoSS
medical-examinations_DmeG
agonist-of_CaoG
first-aid-measures_CHfamC
side-effect_CseD
has-parts-of-the-class_GThpotcGT
biological-process_GbpD
decays-to_CdtC
part-of_DpoD
medical-condition_CmcD
part-of_PDpoSS
afflicts_CHaG
ha

Filtered with a cutoff of 0.01, there still a lot of edge types at 59, and several with 1 or few instances

In [25]:
print_cutoff_info(p05, '0.05', p01, '0.01')

--0.05--
Total Number of Edge Types: 47
Edge Types with only 1 instance: 0
Edge Types with 10 or fewer instances: 3


--------------------------------------------------
This cutoff removes the following edges that were in 0.01:
--------------------------------------------------
part-of_BSpoSM
afflicts_CHaPF
part-of_SSpoSM
cause-of-death_CHcodD
has-part_SShpBS
subclass-of_SsoS
instance-of_CioC
instance-of_SioGT
subclass-of_SsoD
cause-of_ScoC
has-part_ChpC
subclass-of_GsoC


In [26]:
print_cutoff_info(p1, '0.1', p05, '0.05')

--0.1--
Total Number of Edge Types: 44
Edge Types with only 1 instance: 0
Edge Types with 10 or fewer instances: 3


--------------------------------------------------
This cutoff removes the following edges that were in 0.05:
--------------------------------------------------
drug-used-for-treatment_CHduftC
chromosome_MMcGT
chromosome_GcGT


In [27]:
print_cutoff_info(p15, '0.15', p1, '0.1')

--0.15--
Total Number of Edge Types: 42
Edge Types with only 1 instance: 0
Edge Types with 10 or fewer instances: 2


--------------------------------------------------
This cutoff removes the following edges that were in 0.1:
--------------------------------------------------
part-of_PDpoPD
subclass-of_DsoS


In [28]:
print_cutoff_info(p2, '0.2', p15, '0.15')

--0.2--
Total Number of Edge Types: 37
Edge Types with only 1 instance: 0
Edge Types with 10 or fewer instances: 0


--------------------------------------------------
This cutoff removes the following edges that were in 0.15:
--------------------------------------------------
cause-of_CHcoD
has-cause_ShcD
physically-interacts-with_GpiwC
cause-of_DcoS
part-of_GTpoGT
regulates-molecular-biology_GTrmbGT


In [29]:
print_cutoff_info(p25, '0.25', p2, '0.2')

--0.25--
Total Number of Edge Types: 37
Edge Types with only 1 instance: 0
Edge Types with 10 or fewer instances: 0


--------------------------------------------------
This cutoff removes the following edges that were in 0.2:
--------------------------------------------------


In [30]:
print_cutoff_info(p3, '0.3', p25, '0.25')

--0.3--
Total Number of Edge Types: 35
Edge Types with only 1 instance: 0
Edge Types with 10 or fewer instances: 0


--------------------------------------------------
This cutoff removes the following edges that were in 0.25:
--------------------------------------------------
subclass-of_PDsoPD
genetic-association_DgaG


Lose the edge Gene-genetic-association->Disease at a cutoff of .3. This is a very important edge, so must stay below that

In [31]:
p1['e_type'].value_counts()

regulates-molecular-biology_MMrmbG     277911
biological-process_GbpGT               138665
cell-component_GccGT                    87112
molecular-function_GmfGT                69097
has-part_GhpPD                          41086
subclass-of_GTsoGT                      20781
subclass-of_GsoPF                       10697
part-of_PDpoPF                           7514
subclass-of_DsoD                         7011
instance-of_GTioGT                       6403
part-of_GTpoGT                           4234
subclass-of_PFsoPF                       3689
physically-interacts-with_GpiwC          3605
has-part_GhpSS                           3542
regulates-molecular-biology_GTrmbGT      3287
drug-used-for-treatment_DduftC           2651
genetic-association_DgaG                 2091
has-part_GhpSM                           2023
subclass-of_PDsoPD                       1578
part-of_SSpoPF                           1319
significant-drug-interaction_CsdiC        995
symptoms_CHsD                     

In [32]:
p2['e_type'].value_counts()

regulates-molecular-biology_MMrmbG    277911
biological-process_GbpGT              138665
cell-component_GccGT                   87112
molecular-function_GmfGT               69097
has-part_GhpPD                         41086
subclass-of_GTsoGT                     20781
subclass-of_GsoPF                      10697
part-of_PDpoPF                          7514
subclass-of_DsoD                        7011
instance-of_GTioGT                      6403
subclass-of_PFsoPF                      3689
physically-interacts-with_CpiwG         3605
has-part_GhpSS                          3542
drug-used-for-treatment_DduftC          2651
genetic-association_DgaG                2091
has-part_GhpSM                          2023
subclass-of_PDsoPD                      1578
part-of_SSpoPF                          1319
significant-drug-interaction_CsdiC       995
symptoms_CHsD                            984
has-part_GhpAS                           946
has-part_GhpBS                           786
symptoms_C

In [13]:
prepped, node_types, reciprocals = p2, nt, r2

In [14]:
prepped.to_hdf('data/prepped_edges.h5', 'edges')
with open('data/node_types.json', 'w') as fout:
    json.dump(node_types, fout, indent=2)

with open('data/reciprocal_relations.txt', 'w') as fout2:    
    for relations in reciprocals:
        for rel in relations:
            fout2.write(rel+',')
        fout2.write('\n')

### Format to neo4j and save the files

In [34]:
node_neo = pt.format_nodes_neo(prepped, node_types)
edge_neo = pt.format_edges_neo(prepped)

In [35]:
# How many of each node type is represented?
node_neo[':LABEL'].value_counts()

Gene                               19655
GO Term                            19554
Protein Family                     11058
Disease                             6547
Protein Domain                      5479
Mature MicroRNA                     2582
Compound                            2569
Chemical Hazard                      670
Supersecondary Structure             613
Structural Motif                     126
Active Site                          112
Binding Site                          70
Symptom                               27
Post-translational Modification       15
Name: :LABEL, dtype: int64

In [36]:
node_neo.to_csv('data/hetnet_nodes.csv', index=False)
edge_neo.to_csv('data/hetnet_edges.csv', index=False)

## Make the metagraph needed for the learn pipline

#### See how long of metapaths we should use

In [37]:
met = pt.get_metaedge_tuples(prepped, node_types, reciprocals)
abv = pt.get_abbrev_dict(prepped, node_types)

In [38]:
from hetio.hetnet import MetaGraph

mg = MetaGraph.from_edge_tuples(met, abv)

In [39]:
num4 = len(mg.extract_metapaths('Compound', 'Disease', 4))
num5 = len(mg.extract_metapaths('Compound', 'Disease', 5))
num6 = len(mg.extract_metapaths('Compound', 'Disease', 6))

for pl, paths in zip([4, 5, 6], [num4, num5, num6]):
    print('For a max path length of {}, there are {} metapaths'.format(pl, paths))

For a max path length of 4, there are 175 metapaths
For a max path length of 5, there are 887 metapaths
For a max path length of 6, there are 4883 metapaths


In [40]:
with open('len4.txt', 'w') as fout:
    for mp in mg.extract_metapaths('Compound', 'Disease', 4):
        fout.write(str(mp)+'\n')
        
with open('len5.txt', 'w') as fout:
    for mp in mg.extract_metapaths('Compound', 'Disease', 5):
        fout.write(str(mp)+'\n')
        
with open('len6.txt', 'w') as fout:
    for mp in mg.extract_metapaths('Compound', 'Disease', 6):
        fout.write(str(mp)+'\n')

## Metagraph Analysis

Metapaths of length 4:
- 175 pathtypes
- All node types are included
- Many Node types have only 1 associated Metapath
 
Metapaths of length 5:
- 877 pathtypes
- All Node Types represented
- Many paths for each node type

Metapaths of length 6:
- 4,833 pathtypes
- Lots more examples here, would be great if we had the computation

## Make Hetio information

In [41]:
node_neo[':LABEL'].value_counts()

Gene                               19655
GO Term                            19554
Protein Family                     11058
Disease                             6547
Protein Domain                      5479
Mature MicroRNA                     2582
Compound                            2569
Chemical Hazard                      670
Supersecondary Structure             613
Structural Motif                     126
Active Site                          112
Binding Site                          70
Symptom                               27
Post-translational Modification       15
Name: :LABEL, dtype: int64

In [42]:
pt.prep_hetio(prepped, node_types, reciprocals)

Added 69077 nodes to graph
Added 693773 edges to graph


# Metagraph after the protein-gene merge

![New Metagraph](wikidata-metagraph-genemerge-0.2-cutoff.png)